In [ ]:
# Install the transformers library
!pip install transformers datasets
!pip install --upgrade accelerate

In [ ]:
!python -m pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Import required packages
import torch
import pandas as pd
import numpy as np

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="andreas122001/roberta-mixed-detector")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/885 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

tokenizer = AutoTokenizer.from_pretrained("andreas122001/roberta-mixed-detector")
model = AutoModelForSequenceClassification.from_pretrained("andreas122001/roberta-mixed-detector")

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts["input_ids"])

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [ ]:
# Load tokenizer and model, create trainer
trainer = Trainer(model=model)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("NicolaiSivesind/human-vs-machine", 'wiki_labeled')

Generating train split:   0%|          | 0/210000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/45000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/45000 [00:00<?, ? examples/s]

In [ ]:
pred_dataset_label = load_dataset("NicolaiSivesind/human-vs-machine", 'research_abstracts_labeled')

Generating train split:   0%|          | 0/14000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
# Example: Import data from csv-file stored on Google Drive

#from google.colab import drive
#drive.mount('/content/drive')


#file_name = "/content/drive/MyDrive/Colab Notebooks/your-filename.csv"
#text_column = "text"

#df_pred = pd.read_csv(file_name)
#pred_texts = df_pred[text_column].dropna().astype('str').tolist()

In [ ]:
dataset["train"]

Dataset({
    features: ['title', 'label', 'text', 'word_count'],
    num_rows: 210000
})

In [ ]:
dataset["test"]

Dataset({
    features: ['title', 'label', 'text', 'word_count'],
    num_rows: 45000
})

In [ ]:
dataset["validation"]

Dataset({
    features: ['title', 'label', 'text', 'word_count'],
    num_rows: 45000
})

In [ ]:
# Create list of texts (can be imported from .csv, .xls etc.)
# pred_texts = ['Automatic synthesis of faces from visual attributes is an important problem in computer vision and has wide applications in law enforcement and entertainment. With the advent of deep generative convolutional neural networks (CNNs), attempts have been made to synthesize face images from attributes and text descriptions. In this paper, we take a different approach, where we formulate the original problem as a stage-wise learning problem. We first synthesize the facial sketch corresponding to the visual attributes and then we reconstruct the face image based on the synthesized sketch. The proposed Attribute2Sketch2Face framework, which is based on a combination of deep Conditional Variational Autoencoder (CVAE) and Generative Adversarial Networks (GANs), consists of three stages: (1) Synthesis of facial sketch from attributes using a CVAE architecture, (2) Enhancement of coarse sketches to produce sharper sketches using a GAN-based framework, and (3) Synthesis of face from sketch using another GAN-based network. Extensive experiments and comparison with recent methods are performed to verify the effectiveness of the proposed attribute-based three stage face synthesis method.',
#              'This paper presents an innovative method for face synthesis from visual attributes using conditional Variational Autoencoders (VAEs) and Generative Adversarial Networks (GANs). The proposed system allows generating high-quality facial images by feeding the model with sketch-based visual attributes as input. Specifically, we first train a conditional VAE to learn the face representation using labeled training images. Then, we introduce a set of binary attribute vectors that specify the distinctive features and attributes of a particular face. Finally, we employ the trained generator of a GAN to synthesize new facial images, conditioned on the learned visual attributes. Our experiments demonstrate that the proposed methodology outperforms other existing methods in terms of facial attribute manipulation, with the added benefit of producing more diverse and realistic outputs. Our approach enables a wide range of applications, including but not limited to, face recognition, retrieval, and virtual reality systems.',
#              'We show that though conformal symmetry can be broken by the dilaton, such can happen without breaking the conformal degeneracy patterns in the spectra. We departure from R^1XS^3 slicing of AdS_5 noticing that the inverse radius, R, of S^3 relates to the temperature of the deconfinement phase transition and has to satisfy, \hbar c/R >> \Lambda_{QCD}. We then focus on the eigenvalue problem of the S^3 conformal Laplacian, given by 1/R^2 (K^2+1), with K^2 standing for the Casimir invariant of the so(4) algebra. Such a spectrum is characterized by a (K+1)^2 fold degeneracy of its levels, with K\in [0,\infty). We then break the conformal S^3 metric as, d\tilde{s}^2=e^{-b\chi} ((1+b^2/4) d\chi^2 +\sin^2\chi (d\theta ^2 +\sin^2\theta d\varphi ^2)), and attribute the symmetry breaking scale, b\hbar^2c^2/R^2, to the dilaton. We show that such a metric deformation is equivalent to a breaking of the conformal curvature of S^3 by a term proportional to b\cot \chi, and that the perturbed conformal Laplacian is equivalent to (\tilde{K}^2 +c_K), with c_K a representation constant, and \tilde{K}^2 being again an so(4) Casimir invariant, but this time in a representation unitarily inequivalent to the 4D rotational. In effect, the spectra before and after the symmetry breaking are determined each by eigenvalues of a Casimir invariant of an so(4) algebra, a reason for which the degeneracies remain unaltered though the conformal group symmetry breaks at the level of the representation of its algebra. We fit the S^3 radius and the \hbar^2c^2b/R^2 scale to the high-lying excitations in the spectra of the unflavored mesons, and observe the correct tendency of the \hbar c /R=373 MeV value to notably exceed \Lambda_{QCD}. The size of the symmetry breaking scale is calculated as \hbar c \sqrt{b}/R=673.7 MeV.',
#              'This research investigates the phenomenon of conformal symmetry breaking and its correlation with the degeneracy of high-lying unflavored mesons. Conformal symmetry, the invariance of a theory under changes of scale, has long been considered a fundamental tool for understanding high-energy physics. However, in recent years, it has become increasingly apparent that this symmetry is often broken, even in theories where conventional wisdom suggests it should be exact. A prominent example is QCD, the theory of the strong interaction, whose conformal symmetry is well-known to be broken at low energies. We focus on the impact of conformal symmetry breaking on the mass spectrum of high-lying unflavored mesons, a subject that has received comparatively little attention in the literature. We show that the presence of broken conformal symmetry can lead to splitting of the meson masses, even in the absence of explicit chiral symmetry breaking. Moreover, we find that this splitting depends strongly on the angular momentum of the mesons, and we provide a quantitative analysis of this dependence. Our results have important implications for the interpretation of meson spectroscopy data, as well as for the construction of effective theories of QCD. For instance, we discuss how our findings may help to resolve long-standing puzzles in the observed spectrum of mesons, and we propose a framework for systematically incorporating conformal symmetry breaking into calculations of meson properties. Furthermore, we highlight the potential relevance of our work for broader questions in high-energy physics, such as the study of holographic dualities and the search for new phenomena beyond the Standard Model. To summarize, our research sheds light on the interplay between conformal symmetry breaking and meson degeneracy, revealing new features of high-energy physics that were previously overlooked. Our results provide a quantitative understanding of how broken conformal symmetry affects the meson spectrum, and suggest novel directions for future investigations.',
#              'This paper exhibits the closed-loop design constraints using the non-analytic function theory. First, the paper generalizes the sensitivity integral for linear feedback systems with the non-analytic sensitivity function. Sensitivity inequalities are determined by the integral relationships based on the presence of non-minimum phase zeros and right half plane poles. These inequalities are rephrased in plant parameter context, which must be satisfied by the feedback design. That indicates the ability of controllers under the influence of input disturbances and plant parameter variations. The paper then extends the integral to the analytic sensitivity function of the augmented linear feedback systems. This is useful to augment the ability of a linear feedback system to handle input disturbances and plant uncertainties, via modified sensitivity function theory. Numerical simulations are carried out to perform sensitivity analysis on three chemical control systems. That describes the usefulness and demonstrates the applicability of the result of this paper to examine and augment the ability of linear feedback system.',
#              "This paper presents sensitivity integrals and related inequalities to enhance the performance and stability of process control systems. Sensitivity integrals quantify the effects of disturbances on the output of control systems, providing important insights into the system's behavior. By using these integrals, we derive inequalities that impose bounds on the sensitivity integrals, resulting in improved control system robustness and better tracking of desired set-points. Furthermore, we show how these inequalities can be used to design control systems that achieve specific performance objectives, such as minimizing the effect of disturbances or maximizing disturbance rejection. The proposed methodology is illustrated through several examples, demonstrating its effectiveness in practical settings. The results suggest that sensitivity integrals and related inequalities are valuable tools for engineers and researchers working in the field of process control, offering a powerful approach to enhance the performance and robustness of control systems."]

In [ ]:
tokenized_texts = tokenizer(pred_dataset_label['train']['text'],truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [ ]:
# Tokenize texts and create prediction data set
#tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
#pred_dataset = SimpleDataset(tokenized_texts)

In [ ]:
# Run predictions
predictions = trainer.predict(pred_dataset)

In [ ]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [ ]:
# Create DataFrame with texts, predictions, labels, and scores
df = pd.DataFrame(list(zip(pred_dataset_label['train']['text'],preds,labels,scores)), columns=['text','pred','label','score'])
df

,text,pred,label,score
0,Coupling losses were studied in composite tape...,0,human-produced,0.998890
1,"In this study, we investigate the coupling los...",1,machine-generated,0.999882
2,Let $\mathsf M_{\mathsf S}$ denote the strong ...,0,human-produced,0.999847
3,"In this paper, we investigate Weighted Solyani...",1,machine-generated,0.999900
4,In 2019 October Betelgeuse began a decline in ...,1,machine-generated,0.999931
...,...,...,...,...
13995,The coherent transport of magnetic information...,1,machine-generated,0.999895
13996,We present a study of the infrared properties ...,0,human-produced,0.999835
13997,This paper presents the results of a study of ...,1,machine-generated,0.999910
13998,Two tripartite schemes are put forward with sh...,0,human-produced,0.996711


In [ ]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(pred_dataset_label['train']['label'], df['pred'])
recall = recall_score(pred_dataset_label['train']['label'], df['pred'])

In [ ]:
print("Precision: ", precision)
print("Recall: ", recall)

Precision:  0.7301653757588444
Recall:  0.9965714285714286


In [ ]:
import pickle
# Dump the trained Random Forest classifier with Pickle
rbt_pkl_filename = 'HuggingFace_Roberta_GenAI.pkl'
# Open the file to save as pkl file
rbt_Model_pkl = open(rbt_pkl_filename, 'wb')
pickle.dump(trainer, rbt_Model_pkl)
# Close the pickle instances
rbt_Model_pkl.close()